# ヤコビアン

In [1]:
import sympy as sy
from sympy.printing.numpy import NumPyPrinter
from sympy import julia_code
from sympy.utilities.codegen import codegen
import tqdm
import os
from pathlib import Path

from kinematics import Global

In [2]:
N=5
kinem = Global(N)
kinem.set_J_v_simple()

computing kinematics...
Matrix([[q_large[0, 0]], [q_large[1, 0]], [q_large[2, 0]]])
Matrix([[q_large[3, 0]], [q_large[4, 0]], [q_large[5, 0]]])
Matrix([[q_large[6, 0]], [q_large[7, 0]], [q_large[8, 0]]])
Matrix([[q_large[9, 0]], [q_large[10, 0]], [q_large[11, 0]]])
Matrix([[q_large[12, 0]], [q_large[13, 0]], [q_large[14, 0]]])
done computing kinematics!


ヤコビアンを時間微分

コード生成

In [3]:
# Cコード生成
cwd = str(Path().resolve())
base = cwd + "/derived"
dir_name = base + '/c_src/'
os.makedirs(dir_name, exist_ok=True)

xi = sy.Symbol("xi")

def gen_c(f, name, dir_name):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, f),
        language="C",
        project= name + "project",
        to_files=False
    )
    
    f = open(dir_name + c_name, 'w')
    f.write(c_code)
    f.close()

    f = open(dir_name + h_name, 'w')
    f.write(c_header)
    f.close()


os.makedirs(dir_name + "Phi_s/", exist_ok=True)
for i, Phi in enumerate(kinem.Phi_s):
    gen_c(Phi.subs(kinem.xi_large[i, 0], xi), "Phi_"+str(i), dir_name + "Phi_s/")


os.makedirs(dir_name + "J_s/", exist_ok=True)
for i, J in enumerate(kinem.J_v_simples):
    gen_c(J.subs(kinem.xi_large[i, 0], xi), "J_"+str(i), dir_name + "J_s/")



In [4]:
# # juliaコード生成
# dir_name = base + '/julia_src'
# os.makedirs(dir_name + "/Phi", exist_ok=True)
# os.makedirs(dir_name + "/J", exist_ok=True)

# julia_word = "function f(q_large::Matrix{T}, xi_large::Matrix{T}) where T\n    "

# for i, P in enumerate(kinem.Phi_s):
#     name = dir_name + "/Phi" + "/Phi_" + str(i) + ".jl"
#     f = open(name, 'w')
#     f.write("module " + "Phi_" + str(i) + "\n")
#     f.write(julia_word)
#     f.write(julia_code(P))
#     f.write("\nend\nend")
#     f.close()


# for i, J in enumerate(kinem.J_v_simples):
#     name = dir_name + "/J" + "/J_" + str(i) + ".jl"
#     f = open(name, 'w')
#     f.write("module " + "J_" + str(i) + "\n")
#     f.write(julia_word)
#     f.write(julia_code(J))
#     f.write("\nend\nend")
#     f.close()